In [8]:
import glob,sys
from pathlib import Path
pinkRig_path= glob.glob(r'C:\Users\*\Documents\Github\PinkRigs')
pinkRig_path = Path(pinkRig_path[0])
sys.path.insert(0, (pinkRig_path.__str__()))
kilofolder = __import__("Analysis.+kilo.python_.convert_to_ibl_format")
kilo = getattr(kilofolder, "+kilo")
add_anat_to_ibl_format = kilo.python_.convert_to_ibl_format.add_anat_to_ibl_format

# Path to raw ephys data
#ephys_path = Path(r'Z:\AV014\2022-07-07\ephys\AV014_2022-07-07_NatImagesSparseNoise_g0\AV014_2022-07-07_NatImagesSparseNoise_g0_imec0')
     
#add_anat_to_ibl_format(ephys_path,ks_folder='pyKS',recompute=True)